# Feedbacks CERES via SW fluxes

(1) Reconstruit les données avec les PCs et EOFs des flux SW. 

(2) Multi Linéaire Régression des PCs sur les feedbacks pour trouver les coefficients Beta. 

(3) Retrouver les PCs des observations CERES. 

(4) Prédire les feedbacks des observations.

# Importe modules

In [1]:
# Computational modules 
%matplotlib inline
import xarray as xr
import glob
import os
import numpy as np
import netCDF4
from netCDF4 import Dataset
import pandas as pd
import re
from array import array
from pylab import *
#import geopandas
from eofs.xarray import Eof
from eofs.multivariate.standard import MultivariateEof

# Plotting modules 
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pandas.plotting
import matplotlib.ticker as ticker
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import BoundaryNorm

# Scikit-learn
from sklearn import linear_model
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.neural_network import MLPRegressor

# Functions definition

In [2]:
import sys
sys.path.append('/data/home/globc/peatier/CNRMppe')
import Fonctions
from Fonctions import get_wavg_budget_df
from Fonctions import wavg 
from Fonctions import plotlines_Xdf
from Fonctions import plotlines_1df
from Fonctions import Deltas_Lambda
from Fonctions import get_3D_budget_xarr
from Fonctions import get_3D_xarr
from Fonctions import get_3D_SW_xarr
from Fonctions import get_3D_LW_xarr

# EOF, LW_mean and PCs

In [3]:
LW_mean =  pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_LW_mean.npy').to_xarray().to_array()
LW_mean = LW_mean[0,:,:].rename({'variable':'mode'})
LW_mean

<xarray.DataArray (lat: 128, lon: 256)>
array([[141.47495, 141.47495, 141.47495, ..., 141.47495, 141.47495,
        141.47495],
       [144.49937, 144.49937, 144.49937, ..., 144.49937, 144.49937,
        144.49937],
       [145.12088, 145.12088, 145.12088, ..., 145.85733, 145.12088,
        145.12088],
       ...,
       [189.56772, 189.56772, 189.56772, ..., 189.64738, 189.56772,
        189.56772],
       [189.52794, 189.52794, 189.52794, ..., 189.52794, 189.52794,
        189.52794],
       [187.78711, 187.78711, 187.78711, ..., 187.78711, 187.78711,
        187.78711]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon      (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
    mode     <U7 'SW_mean'

In [4]:
LW_anom =  pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_LW_anom.npy').to_xarray().to_array()
LW_anom = LW_anom[0,:,:].rename({'variable':'mode'})
LW_anom

<xarray.DataArray (time: 104, lat: 128, lon: 256)>
array([[[-0.15753174, -0.15753174, -0.15753174, ..., -0.15753174,
         -0.15753174, -0.15753174],
        [-0.93562317, -0.93562317, -0.93562317, ..., -0.93562317,
         -0.93562317, -0.93562317],
        [-1.183258  , -1.183258  , -1.183258  , ..., -1.1234894 ,
         -1.183258  , -1.183258  ],
        ...,
        [ 2.3677979 ,  2.3677979 ,  2.3677979 , ...,  2.3114777 ,
          2.3677979 ,  2.3677979 ],
        [ 2.5720673 ,  2.5720673 ,  2.5720673 , ...,  2.5720673 ,
          2.5720673 ,  2.5720673 ],
        [ 2.7663422 ,  2.7663422 ,  2.7663422 , ...,  2.7663422 ,
          2.7663422 ,  2.7663422 ]],

       [[-0.25358582, -0.25358582, -0.25358582, ..., -0.25358582,
         -0.25358582, -0.25358582],
        [-1.3831329 , -1.3831329 , -1.3831329 , ..., -1.3831329 ,
         -1.3831329 , -1.3831329 ],
        [-2.0042114 , -2.0042114 , -2.0042114 , ..., -1.8739624 ,
         -2.0042114 , -2.0042114 ],
        ...,
        [ 2.0300446 ,  2.0300446 ,  2.0300446 , ...,  2.0138397 ,
          2.0300446 ,  2.0300446 ],
        [ 2.4886017 ,  2.4886017 ,  2.4886017 , ...,  2.4886017 ,
          2.4886017 ,  2.4886017 ],
        [ 3.1971283 ,  3.1971283 ,  3.1971283 , ...,  3.1971283 ,
          3.1971283 ,  3.1971283 ]],

       [[ 2.4591675 ,  2.4591675 ,  2.4591675 , ...,  2.4591675 ,
          2.4591675 ,  2.4591675 ],
        [ 2.7849884 ,  2.7849884 ,  2.7849884 , ...,  2.7849884 ,
          2.7849884 ,  2.7849884 ],
        [ 3.723114  ,  3.723114  ,  3.723114  , ...,  3.5867767 ,
          3.723114  ,  3.723114  ],
        ...,
        [-0.7435913 , -0.7435913 , -0.7435913 , ..., -0.75328064,
         -0.7435913 , -0.7435913 ],
        [-0.5712433 , -0.5712433 , -0.5712433 , ..., -0.5712433 ,
         -0.5712433 , -0.5712433 ],
        [-0.58695984, -0.58695984, -0.58695984, ..., -0.58695984,
         -0.58695984, -0.58695984]],

       ...,

       [[ 0.9213867 ,  0.9213867 ,  0.9213867 , ...,  0.9213867 ,
          0.9213867 ,  0.9213867 ],
        [-0.5202942 , -0.5202942 , -0.5202942 , ..., -0.5202942 ,
         -0.5202942 , -0.5202942 ],
        [-0.9635925 , -0.9635925 , -0.9635925 , ..., -1.0771484 ,
         -0.9635925 , -0.9635925 ],
        ...,
        [-1.6254425 , -1.6254425 , -1.6254425 , ..., -1.6356049 ,
         -1.6254425 , -1.6254425 ],
        [-1.7259827 , -1.7259827 , -1.7259827 , ..., -1.7259827 ,
         -1.7259827 , -1.7259827 ],
        [-1.4056244 , -1.4056244 , -1.4056244 , ..., -1.4056244 ,
         -1.4056244 , -1.4056244 ]],

       [[-1.5286255 , -1.5286255 , -1.5286255 , ..., -1.5286255 ,
         -1.5286255 , -1.5286255 ],
        [-2.1253815 , -2.1253815 , -2.1253815 , ..., -2.1253815 ,
         -2.1253815 , -2.1253815 ],
        [-3.006485  , -3.006485  , -3.006485  , ..., -2.7653198 ,
         -3.006485  , -3.006485  ],
        ...,
        [-0.93640137, -0.93640137, -0.93640137, ..., -1.0320587 ,
         -0.93640137, -0.93640137],
        [-1.9666443 , -1.9666443 , -1.9666443 , ..., -1.9666443 ,
         -1.9666443 , -1.9666443 ],
        [-2.2373047 , -2.2373047 , -2.2373047 , ..., -2.2373047 ,
         -2.2373047 , -2.2373047 ]],

       [[-1.7113342 , -1.7113342 , -1.7113342 , ..., -1.7113342 ,
         -1.7113342 , -1.7113342 ],
        [-2.4022675 , -2.4022675 , -2.4022675 , ..., -2.4022675 ,
         -2.4022675 , -2.4022675 ],
        [-2.959198  , -2.959198  , -2.959198  , ..., -2.9439545 ,
         -2.959198  , -2.959198  ],
        ...,
        [ 2.1657562 ,  2.1657562 ,  2.1657562 , ...,  2.1085663 ,
          2.1657562 ,  2.1657562 ],
        [ 1.6786346 ,  1.6786346 ,  1.6786346 , ...,  1.6786346 ,
          1.6786346 ,  1.6786346 ],
        [ 1.339447  ,  1.339447  ,  1.339447  , ...,  1.339447  ,
          1.339447  ,  1.339447  ]]], dtype=float32)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 ... 95 96 97 98 99 100 101 102 103
  * lat      (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 

In [5]:
eof1 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap1_LW.npy').to_xarray().to_array()
eof2 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap2_LW.npy').to_xarray().to_array()
eof3 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap3_LW.npy').to_xarray().to_array()
eof4 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap4_LW.npy').to_xarray().to_array()
eof5 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap5_LW.npy').to_xarray().to_array()

eof1 = eof1[0,:,:].rename({'variable':'mode'})
eof2 = eof2[0,:,:].rename({'variable':'mode'})
eof3 = eof3[0,:,:].rename({'variable':'mode'})
eof4 = eof4[0,:,:].rename({'variable':'mode'})
eof5 = eof5[0,:,:].rename({'variable':'mode'})

eofs = [eof1,eof2,eof3,eof4,eof5]

In [6]:
eofs_combined = xr.combine_nested(eofs, concat_dim='eofs')
#eofs_combined

In [7]:
pc1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF1pc_LW.npy')
pc2 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF2pc_LW.npy')
pc3 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF3pc_LW.npy')
pc4 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF4pc_LW.npy')
pc5 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF5pc_LW.npy')

PC_df = pd.DataFrame(pc1)
PC_df['pc1'] = pc1
PC_df['pc2'] = pc2
PC_df['pc3'] = pc3
PC_df['pc4'] = pc4
PC_df['pc5'] = pc5
PC_df = PC_df.drop(labels = 0, axis=1)
#PC_df

PC_df

,pc1,pc2,pc3,pc4,pc5
0,-0.739219,-1.244866,0.231087,-0.114967,-2.073620
1,0.336901,0.330346,1.022992,0.410326,0.460474
2,1.772028,0.158484,0.389880,0.341204,-0.621712
3,0.602520,-0.166455,1.601341,-0.169788,0.693402
4,0.675742,-0.001736,-0.160190,1.804740,-0.257126
...,...,...,...,...,...
97,-0.303779,0.396202,1.087760,0.318875,0.982107
98,-0.210095,1.022319,-1.029760,1.346775,1.742032
99,0.184986,0.194929,-0.455570,0.427989,0.221776
100,0.761945,0.240466,0.032001,0.808282,-0.055405


In [8]:
pc1_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF1pc_LW.npy')
pc2_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF2pc_LW.npy')
pc3_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF3pc_LW.npy')
pc4_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF4pc_LW.npy')
pc5_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF5pc_LW.npy')

pc1_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF1pc_LW.npy')
pc2_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF2pc_LW.npy')
pc3_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF3pc_LW.npy')
pc4_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF4pc_LW.npy')
pc5_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF5pc_LW.npy')

In [9]:
PCs = [0]*102
for i in range(0,102,1):
    tmp = PC_df.iloc[i].values
    PCs[i] = tmp

PCs

[array([-0.73921901, -1.24486621,  0.23108684, -0.11496683, -2.0736196 ]),
 array([0.33690146, 0.33034635, 1.02299249, 0.41032562, 0.46047366]),
 array([ 1.77202821,  0.15848448,  0.38987991,  0.34120384, -0.62171173]),
 array([ 0.60251993, -0.16645463,  1.60134053, -0.16978788,  0.69340181]),
 array([ 6.75741673e-01, -1.73643337e-03, -1.60190359e-01,  1.80473983e+00,
        -2.57125735e-01]),
 array([ 0.84083104,  0.50359117, -0.12907813, -0.18733302, -0.27295816]),
 array([ 0.7647841 , -2.41628299,  0.23717833, -0.28449625, -0.92981225]),
 array([ 0.51603752,  0.01315336, -0.32240689, -0.96591097,  0.2885291 ]),
 array([ 0.61629099, -1.37954222, -0.67433822, -1.0695827 , -0.39938933]),
 array([-0.29589611,  0.37804902,  0.86703819, -1.94815981,  0.18674015]),
 array([ 0.5475291 , -0.63170472, -0.12040826, -0.09554155,  0.27197084]),
 array([-0.32486817, -0.57406539,  1.40010118, -1.07604861,  0.71255744]),
 array([-0.07113136,  1.6846796 ,  1.01230752,  0.12115594,  0.1154781 ]),
 a

# Trouver U_obs : Reconstruciton des données

X_ens = U_ens(pc) * v_ens(eof) 

Compute anomaly : X_obs = X_obs - SW_mean_ens 

X_obs = U_obs * v_ens 

U_obs = X_obs * inv(v_ens)

## Get the CMIP6 datatset

In [10]:
def load_monthly_clim(path, filename, variables) :
    
    file = xr.open_mfdataset(path+filename,combine='by_coords')
    df=file[variables].to_dataframe()
    
    # Compute Climatological Annual Cycle :
    df1=df.reset_index(level=['time', 'lat', 'lon'])
    df1['month']=pd.DatetimeIndex(df1['time']).month # time in years
    df_mean=df1.groupby(['month', 'lat', 'lon']).mean()
    
    return df_mean

In [11]:
# Download the amip dataframes and compute the radiative budget  

path = "/data/scratch/globc/peatier/CMIP6/CNRM-CM6-1/CFMIP/amip/"
filename = "*_amip_*.nc"
variables = ['rlut']
CNRM_amip = load_monthly_clim(path, filename, variables)

path = "/data/scratch/globc/peatier/CMIP6/MRI-ESM2-0/amip/"
MRI_amip = load_monthly_clim(path, filename, variables)

path = "/data/scratch/globc/peatier/CMIP6/CanESM5/amip/"
filename = "standard_*_amip_*regrid.nc"
CAN_amip = load_monthly_clim(path, filename, variables)
CAN_amip

path = "/data/scratch/globc/peatier/CMIP6/MIROC6/"
filename = "*_amip_*regrid.nc"
MIROC_amip = load_monthly_clim(path, filename, variables)
MIROC_amip

path = "/data/scratch/globc/peatier/CMIP6/HadGEM3-GC31-LL/"
filename = "standard_*_amip_*regrid.nc"
HadGEM3_amip = load_monthly_clim(path, filename, variables)
HadGEM3_amip

path = "/data/scratch/globc/peatier/CMIP6/IPSL-CM6A-LR/"
filename = "*_amip_*regrid.nc"
IPSL_amip = load_monthly_clim(path, filename, variables)
IPSL_amip

path = "/data/scratch/globc/peatier/CMIP6/BCC-CSM2-MR/"
filename = "standard_*_amip_*regrid.nc"
BCC_amip = load_monthly_clim(path, filename, variables)
BCC_amip

rlut  height
month lat        lon                          
1     -88.927735 0.00000    185.960541     2.0
                 1.40625    185.925110     2.0
                 2.81250    185.888687     2.0
                 4.21875    185.853851     2.0
                 5.62500    185.800659     2.0
...                                ...     ...
12     88.927735 352.96875  165.538513     2.0
                 354.37500  165.532074     2.0
                 355.78125  165.529419     2.0
                 357.18750  165.519089     2.0
                 358.59375  165.541138     2.0

[393216 rows x 2 columns]

In [12]:
df_obs = pd.DataFrame()
df_obs['CNRM_rlut'] = CNRM_amip['rlut']
df_obs['MRI_rlut'] = MRI_amip['rlut']
df_obs['CAN_rlut'] = CAN_amip['rlut']
df_obs['MIROC_rlut'] = MIROC_amip['rlut']
df_obs['HadGEM3_rlut'] = HadGEM3_amip['rlut']
df_obs['IPSL_rlut'] = IPSL_amip['rlut']
df_obs['BCC_rlut'] = BCC_amip['rlut']
#df_obs = df_obs.drop(['rsut', 'height'], axis = 1)
df_obs

CNRM_rlut    MRI_rlut    CAN_rlut  MIROC_rlut  \
month lat        lon                                                         
1     -88.927735 0.00000    185.312866  191.050308  194.395706  199.672424   
                 1.40625    185.312866  190.944885  194.394623  199.639511   
                 2.81250    185.312866  190.871658  194.044556  199.592834   
                 4.21875    185.312866  190.784973  194.043549  199.539337   
                 5.62500    185.312866  190.762955  194.042526  199.481033   
...                                ...         ...         ...         ...   
12     88.927735 352.96875  172.606918  169.241440  165.647949  163.121719   
                 354.37500  172.606918  169.257797  165.759399  163.137863   
                 355.78125  172.606918  169.250305  165.759644  163.157043   
                 357.18750  172.606918  169.255676  165.863510  163.157791   
                 358.59375  172.606918  169.260330  165.863800  163.163940   

                            HadGEM3_rlut   IPSL_rlut    BCC_rlut  
month lat        lon                                              
1     -88.927735 0.00000      192.111023  189.196426  185.960541  
                 1.40625      192.087799  189.144714  185.925110  
                 2.81250      192.078903  189.089417  185.888687  
                 4.21875      192.046692  189.021576  185.853851  
                 5.62500      192.015869  188.955856  185.800659  
...                                  ...         ...         ...  
12     88.927735 352.96875    168.190277  177.472336  165.538513  
                 354.37500    168.202057  177.507263  165.532074  
                 355.78125    168.214233  177.521210  165.529419  
                 357.18750    168.229095  177.518356  165.519089  
                 358.59375    168.240646  177.534149  165.541138  

[393216 rows x 7 columns]

In [13]:
obs = df_obs.groupby(['lat','lon']).mean().to_xarray()
obs

<xarray.Dataset>
Dimensions:       (lat: 128, lon: 256)
Coordinates:
  * lat           (lat) float64 -88.93 -87.54 -86.14 ... 86.14 87.54 88.93
  * lon           (lon) float64 0.0 1.406 2.812 4.219 ... 355.8 357.2 358.6
Data variables:
    CNRM_rlut     (lat, lon) float32 141.31741 141.31741 ... 190.55353 190.55353
    MRI_rlut      (lat, lon) float32 144.62071 144.55673 ... 191.9757 191.97809
    CAN_rlut      (lat, lon) float32 145.37804 145.37663 ... 190.26115 190.2614
    MIROC_rlut    (lat, lon) float32 145.74411 145.68283 ... 186.1432 186.14372
    HadGEM3_rlut  (lat, lon) float32 142.43832 142.40167 ... 191.7751 191.78168
    IPSL_rlut     (lat, lon) float32 143.39436 143.30669 ... 196.88176 196.88039
    BCC_rlut      (lat, lon) float32 141.79498 141.73112 ... 190.28763 190.28816

## Compute Anomaly

In [15]:
X_obs = {}
X_obs['CNRM'] = obs['CNRM_rlut'] - LW_mean
X_obs['MRI'] = obs['MRI_rlut'] - LW_mean
X_obs['CAN'] = obs['CAN_rlut'] - LW_mean
X_obs['MIROC'] = obs['MIROC_rlut'] - LW_mean
X_obs['HadGEM3'] = obs['HadGEM3_rlut'] - LW_mean
X_obs['IPSL'] = obs['IPSL_rlut'] - LW_mean
X_obs['BCC'] = obs['BCC_rlut'] - LW_mean
X_obs

{'CNRM': <xarray.DataArray (lat: 128, lon: 256)>
 array([[-0.15753174, -0.15753174, -0.15753174, ..., -0.15753174,
         -0.15753174, -0.15753174],
        [-0.9356384 , -0.9356384 , -0.9356384 , ..., -0.9356384 ,
         -0.9356384 , -0.9356384 ],
        [-1.1833038 , -1.1833038 , -1.1833038 , ..., -1.1234741 ,
         -1.1833038 , -1.1833038 ],
        ...,
        [ 2.3677216 ,  2.3677216 ,  2.3677216 , ...,  2.3114624 ,
          2.3677216 ,  2.3677216 ],
        [ 2.5720825 ,  2.5720825 ,  2.5720825 , ...,  2.5720825 ,
          2.5720825 ,  2.5720825 ],
        [ 2.7664185 ,  2.7664185 ,  2.7664185 , ...,  2.7664185 ,
          2.7664185 ,  2.7664185 ]], dtype=float32)
 Coordinates:
   * lat      (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
   * lon      (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
     mode     <U7 'SW_mean', 'MRI': <xarray.DataArray (lat: 128, lon: 256)>
 array([[3.1457672, 3.081787 , 3.0169678, ..., 3.317459 , 3.2607

## Trouver U_obs

In [16]:
eofs_inv = np.linalg.pinv(eofs_combined.transpose())
#eofs_inv = xr.DataArray(eofs_inv, dims=['eofs','lon','lat'])
eofs_inv

array([[[ 1.5037279e-03,  2.0251980e-03,  2.3831320e-03, ...,
          1.1522119e-04,  1.5181190e-04,  1.0444225e-04],
        [-4.3453327e-03, -5.2094674e-03, -6.0062748e-03, ...,
          1.1676393e-03,  9.7354554e-04,  7.9151511e-04],
        [ 7.8285648e-04,  2.1680163e-03,  2.9920137e-03, ...,
         -3.0047644e-04, -5.3152844e-04, -1.1325346e-03],
        [-3.8616511e-03, -6.1427555e-03, -7.2744559e-03, ...,
         -9.2601553e-03, -8.9076217e-03, -8.5301390e-03],
        [-2.3395126e-04, -7.2320551e-03, -9.0905726e-03, ...,
         -1.0601393e-02, -9.3988236e-03, -7.9502752e-03]],

       [[ 1.5159119e-03,  2.1839852e-03,  2.5626351e-03, ...,
          4.0923068e-04,  4.2583930e-04,  3.6809739e-04],
        [-4.3030940e-03, -5.2683884e-03, -6.0572629e-03, ...,
          9.7447506e-04,  7.9036161e-04,  6.0291099e-04],
        [ 3.8006090e-04,  1.7847304e-03,  2.5630891e-03, ...,
         -5.3573895e-04, -7.8823994e-04, -1.4163568e-03],
        [-3.8680367e-03, -6.0400940e-0

In [17]:
X_CNRM_bis = X_obs['CNRM'].expand_dims(dim={'dummy':range(0,102,1)})
X_MRI_bis = X_obs['MRI'].expand_dims(dim={'dummy':range(0,102,1)})
X_CAN_bis = X_obs['CAN'].expand_dims(dim={'dummy':range(0,102,1)})
X_MIROC_bis = X_obs['MIROC'].expand_dims(dim={'dummy':range(0,102,1)})
X_HadGEM3_bis = X_obs['HadGEM3'].expand_dims(dim={'dummy':range(0,102,1)})
X_IPSL_bis = X_obs['IPSL'].expand_dims(dim={'dummy':range(0,102,1)})
X_BCC_bis = X_obs['BCC'].expand_dims(dim={'dummy':range(0,102,1)})

In [18]:
U_CNRM = np.dot(eofs_inv,X_CNRM_bis.transpose('lon','lat', 'dummy'))
U_MRI = np.dot(eofs_inv,X_MRI_bis.transpose('lon','lat', 'dummy'))
U_CAN = np.dot(eofs_inv,X_CAN_bis.transpose('lon','lat', 'dummy'))
U_MIROC = np.dot(eofs_inv,X_MIROC_bis.transpose('lon','lat', 'dummy'))
U_HadGEM3 = np.dot(eofs_inv,X_HadGEM3_bis.transpose('lon','lat', 'dummy'))
U_IPSL = np.dot(eofs_inv,X_IPSL_bis.transpose('lon','lat', 'dummy'))
U_BCC = np.dot(eofs_inv,X_BCC_bis.transpose('lon','lat', 'dummy'))


In [19]:
U_CNRM = xr.DataArray(U_CNRM, dims=['lon','eofs','lon2', 'dummy'])
U_CNRM_avg = U_CNRM[0,:,0,0].transpose()

U_MRI = xr.DataArray(U_MRI, dims=['lon','eofs','lon2', 'dummy'])
U_MRI_avg = U_MRI[0,:,0,0].transpose()

U_CAN = xr.DataArray(U_CAN, dims=['lon','eofs','lon2', 'dummy'])
U_CAN_avg = U_CAN[0,:,0,0].transpose()

U_MIROC = xr.DataArray(U_MIROC, dims=['lon','eofs','lon2', 'dummy'])
U_MIROC_avg = U_MIROC[0,:,0,0].transpose()

U_HadGEM3 = xr.DataArray(U_HadGEM3, dims=['lon','eofs','lon2', 'dummy'])
U_HadGEM3_avg = U_HadGEM3[0,:,0,0].transpose()

U_IPSL = xr.DataArray(U_IPSL, dims=['lon','eofs','lon2', 'dummy'])
U_IPSL_avg = U_IPSL[0,:,0,0].transpose()

U_BCC = xr.DataArray(U_BCC, dims=['lon','eofs','lon2', 'dummy'])
U_BCC_avg = U_BCC[0,:,0,0].transpose()

In [20]:
#Save the U_obs_avg 
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_CNRM_LW.npy', U_CNRM_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_MRI_LW.npy', U_MRI_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_CAN_LW.npy', U_CAN_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_MIROC_LW.npy', U_MIROC_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_HadGEM3_LW.npy', U_HadGEM3_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_IPSL_LW.npy', U_IPSL_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_BCC_LW.npy', U_BCC_avg)